# Finding Lane Lines on a Road

Sebstien Pignolo _ Udacity Self Driving Car Nanodegree -  December 2016

Objectives:
In this project we try to detect road lane lines through a series of steps forming an image processing pipeline.
At first the pipeline is applied on images to check the validity of the algorithms, and then on videos that are simply a succession of images.

Challenges encountered on videos:
    i)  The "solidWhiteRight.mp4" (960x540px) has left and right lanes in white, some noise exists on the right lane
    like a whitish trace along the right white lane, but the cv2.inRange filters this out. 
    The end of the video is less conclusive as the lines I am drawing become at times parallel with the lane
    and don't superimpose with the road lanes. Taking the average does not allow a quick reaction if the slope changes rapidly. 
    
    ii)  The "solidYellowLeft.mp4" (960x540px) adds a difficulty by adding a yellow lane and some noise
    (some white or yellow elements along the road. The pipeline is still robust.
    At the end of the video the left line is nto following perfectly the yellow line though.
    The pre-defined (using measurement) range of slopes to reject non interesting lines is working most of the time.
    
    iii) The "challenge.mp4" (1280x720px) adds another level of difficulty by adding shadows,
    more noisy white and yellow some elements along the lines, as well as a positioning the camera differently
    (the front of the car appears). The region of interest excludes the front of the car.
    The image color is filtered using cv2.inRange. When the shadow is too deep the lines I am drawing deviate
    from the road lanes but they come backn quick to fit each lane lines.



Pipeline: Four steps form the pipeline;

Stage 1: Obtaining a gray scale image from a BGR picture

    Stage 1-b: add a color filtering for the challenge videos to reject shadows on the yellow lanes

Stage 2: Applying the DeCanny algorithm to detected edges

    

Stage 3: Applying the Hough Transform to find lines, many line segments are found of various slopes

    Stage 3-a: define a region of interest before applying Hough Transform. The rest of the image is 
    Stage 3-b: Keep the lines that are in a certain range of slopes. Right and left lanes have different slopes.

Stage 4: Extrapolate the line segments using Pitagore theoreme 

    Stage 4-b: I end up drawing one line averaging the x and y of the values of all the lines found
    in the range of filtered slopes.
    

Repeat starting Step 1 and write the video output as a succession of images in a mp4 container.


Remarques:
       Remarque 1: The range of slopes for the left and right sides have been found by running the algorithms
       with a wide range at first and then refining the range.
       I started the algorithm with a hardcoded value for the range of slopes, and then I moved to a slighlty more
       adpative filering taking into account the current average slope and allowing a deviation of +/-0.2
       at both side of the current value. 
       The initial value of "starting_slope_right" and "starting_slope_left" are crucial as they decide on the
       initial rejection of calculated slopes. The initial value should not be too far from the true value.
       
       

       Remarque 2: This project was very interesting in the sense that it allowed me to produce the expected results
       with a limited number of lines. I got also to see that many factors can influence the algorithm and that it is
       challenging to produce a robust algorithm that would work in a wide range of conditions (light, noise, curve).

In [35]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
import math
from __future__ import division
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline

starting_slope_right = 0.7
starting_slope_left = -0.7
slope_r = []   # store all the slopes of the right lane to get the average
slope_l = []   # store all the slopes of the left lane to get the average
width = 960 # width of the 2 videos
height = 540 # height of the 2 videos
list_lines_r = []
list_lines_l = []
avg_list_lines = []

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    #return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)


def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines,color=[175, 4, 28], thickness=7)
    return line_img


                
        
def draw_lines(img, lines, color, thickness):#color=[0, 255, 255], thickness=30
    
    global avg_list_lines
    global starting_slope_right
    global starting_slope_left
    
    for line in lines:
        
        for x1,y1,x2,y2 in line:
            #print "starting_slope right", starting_slope_right
            #print "starting_slope left", starting_slope_left
            if ((x2 - x1)!= 0): #to avoid division by zero
            
                #calculate the current slope and compare to a range to reject non interesting lines
                slope_curr = (y2 - y1) / (x2 - x1) 
                if ((slope_curr > (starting_slope_right - 0.1)) and (slope_curr < (starting_slope_right + 0.1))): #adapting the threshold to the current average
                    
                    slope_r.append(slope_curr)
                    slope_avg_r = np.average(slope_r)
                    starting_slope_right = slope_avg_r #adapting the slope to the average right slope
                    
                    # debugging line to fine tune the average slope
                    #print "average slope right lane", slope_avg_r

                    x1e, y1e = extrapolate_line(x1,y1,x2,y2, -width) 
                    x2e, y2e = extrapolate_line(x1,y1,x2,y2, width)  
                    line = np.array([[x1e,y1e,x2e,y2e]])
                    list_lines_r.append(line)
                    
                    
                elif (slope_curr < (starting_slope_left + 0.1) and (slope_curr > (starting_slope_left - 0.1))): #>-0.71 and slope_curr <-0.67):
                                        
                    slope_l.append(slope_curr)
                    slope_avg_l = np.average(slope_l)
                    starting_slope_left = slope_avg_l #adapting the slope to the average left slope

                    #print "average slope left lane", slope_avg_l

                    x1e_l, y1e_l = extrapolate_line(x1,y1,x2,y2, -width) # bottom point
                    x2e_l, y2e_l = extrapolate_line(x1,y1,x2,y2, width)  # top point
                    line = np.array([[x1e_l,y1e_l,x2e_l,y2e_l]])
                    list_lines_l.append(line)

    if len(list_lines_r):
        #selecting the mean of all the non-rejeted lines 
             
        avg_list_lines = np.average(list_lines_r, axis=0)
       
        x11 = int(avg_list_lines[0][0])
        x12 = int(avg_list_lines[0][1])
        x21 = int(avg_list_lines[0][2])
        x22 = int(avg_list_lines[0][3])
        cv2.line(img, (x11,x12), (x21,x22), color, thickness)
        
    
    if len(list_lines_l):
        #selecting the mean of all the non-rejeted lines 

        avg_list_lines_l = np.average(list_lines_l, axis=0)
        x11l = int(avg_list_lines_l[0][0])
        x12l = int(avg_list_lines_l[0][1])
        x21l = int(avg_list_lines_l[0][2])
        x22l = int(avg_list_lines_l[0][3])
        cv2.line(img, (x11l,x12l), (x21l,x22l), color, thickness) 


        
def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
    
  
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def weighted_img(img, initial_img, alpha=0.8, beta=1., delta=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, alpha, img, beta, delta)

def extrapolate_line(x1, y1, x2, y2, height):
    """ Takes line endpoints and extroplates new endpoint
    the region of interest takes care of the part that is to be displayed """
    line_lenght = np.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2) #triangle
    x = x2 + ((x2 - x1) / line_lenght * height)
    y = y2 + ((y2 - y1) / line_lenght * height)
    return int(x), int(y)


def detect_lanes(image):

        #PIPELINE STAGE 1: Obtaining a gray scale image from a BGR picture
    
        # define a range of colors where to look for 200,170,40 is the yellow left lane
        color_low = np.array([220,170,40]) #220,170,40
        color_high = np.array([255,255,255])
        color_mask = cv2.inRange(image, color_low, color_high)

        ## WHYcopy_image = np.copy(image)
        # getting a gray scale of the image
        gray_image = grayscale (image)   
        
        
        #PIPELINE STAGE 2: Applying the DeCanny algorithm to detected edges

        # Define a kernel size for Gaussian smoothing / blurring
        # Canny() applies a 5x5 Gaussian internally but works better with a 3x3
        kernel_size = 3
        blur_gray = cv2.GaussianBlur(gray_image,(kernel_size, kernel_size), 0)
        
        low_threshold = 50
        high_threshold = 150
        edges_image = canny(gray_image, low_threshold, high_threshold)


        #PIPELINE STAGE 3: Applying the Hough Transform to find lines, many line segments are found of various slopes
        
        # Stage 3-a: defining a region of interest to exclude the background and get a focus
        mask_vertices = np.array( [[[0,540],[900,540],[500,320],[460,320],[0,540]]], dtype=np.int32 ) 
        region_of_interest_gray=region_of_interest(edges_image, mask_vertices)


        

        #Parameters for the Hough Transform, found after running several experiments
        rho = 1
        theta = np.pi / 180
        threshold = 1
        min_line_len = 20
        max_line_gap = 20
        line_image = np.copy(edges_image) * 0 #creating a blank to draw lines on      
        line_image= hough_lines(region_of_interest_gray, rho, theta, threshold, min_line_len, max_line_gap)


        #defining a region of interest to exclude the background and give the focus on the laneLines_thirdPass.jpgs
        region_of_interest_gray = region_of_interest(line_image, mask_vertices)

        final_image = weighted_img(region_of_interest_gray, image, alpha=0.8, beta=1., delta=0.)

        
        return final_image


def main ():
    
    white_output = '12_29_output_white_line.mp4'
    clip1 = VideoFileClip("solidWhiteRight.mp4")
    white_clip = clip1.fl_image(detect_lanes) #NOTE: this function expects color images!!
    %time white_clip.write_videofile(white_output, audio=False)
    
    yellow_output = '12_29_output_yellow_line.mp4'
    clip2 = VideoFileClip("solidYellowLeft.mp4")
    yellow_clip = clip2.fl_image(detect_lanes) #NOTE: this function expects color images!!
    %time yellow_clip.write_videofile(yellow_output, audio=False)

    
    
if __name__ == '__main__':
    main()



[MoviePy] >>>> Building video 12_29_output_white_line.mp4
[MoviePy] Writing video 12_29_output_white_line.mp4


100%|█████████▉| 221/222 [00:04<00:00, 52.74it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: 12_29_output_white_line.mp4 

CPU times: user 3.19 s, sys: 1.41 s, total: 4.6 s
Wall time: 4.42 s
[MoviePy] >>>> Building video 12_29_output_yellow_line.mp4
[MoviePy] Writing video 12_29_output_yellow_line.mp4


100%|█████████▉| 681/682 [00:18<00:00, 29.45it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: 12_29_output_yellow_line.mp4 

CPU times: user 16.3 s, sys: 3.89 s, total: 20.2 s
Wall time: 18.8 s
